# GROUP 09 FINAL PROJECT CODE
## This is the code of Newton's Cradle simulation with air particle considered.

Import needed stuffs.

In [ ]:
from vpython import *
from random import random
import numpy as np 

Define constant.

In [ ]:
#const.
g= vector(0,-9.8,0)
[k,r]=[1E5,1.0]
d=0.1
number=1
theta=pi/6
num=2

t = 0
dt=0.001


Create scene and graph.

In [ ]:
#scene
scene=display(width=800,height=800,center=vector(-5*d,r/2,0), background=(0.9,0.9,0.9))

In [ ]:
#graph
gy=graph(width=500,height=300,align='left', ytitle='Height')
gy0=gcurve(graph=gy,color=color.blue, dot=True)
gy1=gcurve(graph=gy,color=color.red, dot=True)

gEk=graph(width=500,height=300,align='left', ytitle='Kinetic Energy')
gEk0=gcurve(graph=gEk,color=color.blue, dot=True)
gEk1=gcurve(graph=gEk,color=color.red, dot=True)

gUg=graph(width=500,height=300,align='left', ytitle='Potential Energy')
gUg0=gcurve(graph=gUg,color=color.blue, dot=True)
gUg1=gcurve(graph=gUg,color=color.red, dot=True)

gE=graph(width=500,height=300,align='left', ytitle='Total Energy')
gE0=gcurve(graph=gE,color=color.blue, dot=True)
gE1=gcurve(graph=gE,color=color.red, dot=True)
gEt=gcurve(graph=gE,color=color.orange, dot=True)


Create objects and particles.

In [ ]:
#create objects
ceiling=box(length=10*d, width=3*d, height=0.01, pos=vector(-5*d,r,0), color=color.blue)
ball=[]
string=[]
wallR = box(pos=vector(-5*d,r+0.5*d,0), size=vector(21*d, 0.2*d, 0.2*d), color=color.green)
wall1 = box(pos=vector(-15.7*d,0.46*r,0), size=vector(0.2*d, 1.2*r, 0.2*d), color=color.green)
wallL = box(pos=vector(-5*d,-1.2*d,0), size=vector(21*d, 0.2*d, 0.2*d), color=color.green)
wall2 = box(pos=vector(5.7*d,0.46*r,0), size=vector(0.2*d, 1.2*r, 0.2*d), color=color.green)
for i in arange(num):
    ball.append(sphere(radius=d,pos=vector(-d-i*2*d,0,0),color=color.red))
    string.append(cylinder(radius=0.005,pos=vec(-d-i*2*d,r,0),axis=vector(0,-r,0)))
    ball[i].v=vector(0,0,0)
    ball[i].m=1

ball[0].m=1
ball[0].color = color.blue
ball[1].color = color.red


In [ ]:
#create particles
N = 200      
L = ((22.4E-3/(6E23))*N)**(1/3.0)/2     
ballsize = 1E-2        
T = 273.15       
t = 0
atoms = []        
vatoms = []
matoms = 0.0004    #28.8/6E23            #重量是重點，可改變去尋找最佳
momentum, vrms =0, (3*k*T/matoms)**0.5  

for i in range(N):              
    position = vector(-15.7*d*random()+5.7*d*random(),r*random()-0.5*d*random(),0)
    atom = sphere(pos=position, radius = ballsize, color=vector(random(), random(), random()))
    atomv = vector(2E-2*random() -2E-2*random(), 2E-2*random()-2E-2*random(), 0) 
    atoms.append(atom)
    vatoms.append(atomv)


Define physical functions.

In [ ]:
#define collision and force function
def newcollision(v1,v2,m1,m2,p1,p2,dis):
    disx=p2.x-p1.x
    disy=p2.y-p1.y
    adis=sqrt(disx**2+disy**2)
    axdis=dis*disx/adis
    aydis=dis*disy/adis
    ax = axdis/dis
    ay = aydis/dis
    vx1=v1.x*ax+v1.y*ay
    vy1=-v1.x*ay+v1.y*ax
    vx2=v2.x*ax+v2.y*ay
    vy2=-v2.x*ay+v2.y*ax
    v1prime = vx1 + (2)*(vx2-vx1)/(1+m1/m2)
    v2prime = vx2 + (2)*(vx1-vx2)/(1+m2/m1)
    v1final = vector(v1prime*ax-vy1*ay,v1prime*ay+vy1*ax,0)
    v2final = vector(v2prime*ax-vy2*ay,v2prime*ay+vy2*ax,0)
    return v1final, v2final
    
def Fs(R):
    return -k*(R-r*norm(R))-ball[i].m*mag(g)*norm(R)


In [ ]:
#define energy function
def E_k(m, v):
    return (1/2)*m*v*v
    
def U_g(m, h):
    return m*mag(g)*h


Pull ball up at the beginning.

In [ ]:
#pull ball up
for i in arange(num):
    if i<number:
        ball[i].pos= string[i].pos+ r*vector(sin(theta),-cos(theta),0)
        string[i].axis= ball[i].pos-string[i].pos


Below is the running loop

In [ ]:
#loop
while t<12:
    rate(10000)
    for i in range(num):
        string[i].axis=ball[i].pos-string[i].pos
        ball[i].v+=(Fs(string[i].axis)/ball[i].m+g)*dt
        ball[i].pos+=ball[i].v*dt
    for i in range(num):
        for j in range(i):
            if (mag(ball[i].pos-ball[j].pos)<2*d and dot((ball[i].pos-ball[j].pos),(ball[i].v-ball[j].v))<0):
                (ball[i].v,ball[j].v)=newcollision(ball[i].v, ball[j].v, ball[i].m, ball[j].m,ball[i].pos, ball[j].pos, 2*d)
    for i in range(N):
        atoms[i].pos += vatoms[i]
    for i in range(N-1):
        for j in range(i,N):
            if (sqrt((atoms[i].pos.x-atoms[j].pos.x)**2 + (atoms[i].pos.y-atoms[j].pos.y)**2)) <= 2*ballsize and dot(atoms[i].pos-atoms[j].pos, vatoms[i]-vatoms[j]) < 0 :
                vatoms[i],vatoms[j] = newcollision(vatoms[i], vatoms[j], matoms, matoms, atoms[i].pos, atoms[j].pos, 2*ballsize)

    for i in range(N):
        if atoms[i].pos.x + ballsize >= 5.7*d-0.2*d+0.01 and atoms[i].pos.x*vatoms[i].x > 0 :     
            atoms[i].pos=vector(-15.7*d*random()+5.7*d*random(),r*random()-0.5*d*random(),0)
            vatoms[i]=vector(2E-2*random() -2E-2*random(), 2E-2*random()-2E-2*random(), 0) 
        else if atoms[i].pos.x - ballsize <= -15.7*d+0.2*d-0.01 and atoms[i].pos.x*vatoms[i].x > 0 :    
            atoms[i].pos=vector(-15.7*d*random()+5.7*d*random(),r*random()-0.5*d*random(),0)
            vatoms[i]=vector(2E-2*random() -2E-2*random(), 2E-2*random()-2E-2*random(), 0)
        else if atoms[i].pos.y + ballsize >= r+0.5*d-0.2*d+0.01 and atoms[i].pos.y*vatoms[i].y > 0 :    
            atoms[i].pos=vector(-15.7*d*random()+5.7*d*random(),r*random()-0.5*d*random(),0)
            vatoms[i]=vector(2E-2*random() -2E-2*random(), 2E-2*random()-2E-2*random(), 0)
        else if atoms[i].pos.y - ballsize <= -1.2*d+0.2*d-0.01 and atoms[i].pos.y*vatoms[i].y > 0 :    
           atoms[i].pos=vector(-15.7*d*random()+5.7*d*random(),r*random()-0.5*d*random(),0)
           vatoms[i]=vector(2E-2*random() -2E-2*random(), 2E-2*random()-2E-2*random(), 0)
    for i in range(num):
        for j in range(N):
            if (sqrt((ball[i].pos.x-atoms[j].pos.x)**2 + (ball[i].pos.y-atoms[j].pos.y)**2)) <= ballsize+d and dot(ball[i].pos-atoms[j].pos, ball[i].v-vatoms[j]) < 0 :
                ball[i].v,vatoms[j] = newcollision(ball[i].v, vatoms[j], ball[i].m, matoms, ball[i].pos, atoms[j].pos, ballsize+d)
    
    #energy
    E_k0= E_k(ball[0].m,mag(ball[0].v))
    E_k1= E_k(ball[1].m,mag(ball[1].v))
    U_g0= U_g(ball[0].m,ball[0].pos.y)
    U_g1= U_g(ball[1].m,ball[1].pos.y)
    E0= E_k0+U_g0
    E1= E_k1+U_g1
    Et= E0+E1
    #plot gy
    gy0.plot(pos=(t,ball[0].pos.y))
    gy1.plot(pos=(t,ball[1].pos.y))
    #plot gEk
    gEk0.plot(pos=(t,E_k0))
    gEk1.plot(pos=(t,E_k1))
    #plot gUg
    gUg0.plot(pos=(t,U_g0))
    gUg1.plot(pos=(t,U_g1))
    #plot gE
    gE0.plot(pos=(t,E0))
    gE1.plot(pos=(t,E1))
    gEt.plot(pos=(t,Et))
    
    t += dt
